In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import regexp_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
# nltk.download('punkt')
from nltk.stem.porter import *
stemmer = PorterStemmer()
from nltk.tokenize import sent_tokenize


headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
}

In [2]:
#xlsx....csv.....float to int
inp = pd.DataFrame(pd.read_excel("D:/Home_Extras/Black_Coffer/InternshipTasks/Assignment/Input.xlsx"))
inp['URL_ID'] = inp['URL_ID'].astype(int)

In [3]:
#Merging all stop words into one txt file
file_names = ['StopWords_Auditor.txt', 'StopWords_Currencies.txt', 'StopWords_DatesandNumbers.txt', 'StopWords_Generic.txt', 'StopWords_GenericLong.txt', 'StopWords_Geographic.txt', 'StopWords_Names.txt']
with open("stw.txt", "w") as outfile:
    for f in file_names:
        with open(f) as infile:
            outfile.write(infile.read())
        outfile.write("\n")    

In [4]:
stopList = open("stw.txt").readlines()
stop_words=[]
for element in stopList:
    stop_words.append(element.strip())
#stop_words

In [5]:
new_inp = inp

In [6]:
new_inp['text'] = ''

In [7]:
#function creates txt file
def createcsv(link, idd_conv,i):
    page = requests.get(link,headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    
    Title= soup.find("title").getText()
    Content= soup.find("div", class_="td-post-content").getText()
    #create new file add title
    f = open(idd_conv,'w', encoding="utf-8")
    f.write(Title)
    f.close()
    #append content to the created file    
    with open(idd_conv,'a', encoding="utf-8") as f:
        f.write(Content)
        f.close()
        
        
#function will add text to new column "text"   
def add_nc(idd_conv,i):    
    with open(idd_conv,'r', encoding="utf-8") as f:    
        new_inp['text'][i] = f.read()
        f.close()
    #check partivular original text from website using print    
#     f = open("1.txt", "r", encoding="utf-8")
#     print(f.read())

for i in range(len(inp["URL_ID"])):
    link = inp["URL"][i]
    idd = inp["URL_ID"][i]
    #variable added to string
    idd_conv = '%s.txt' % idd
    #print(idd_conv)
    #commenting cuz aleardy done  
    #createcsv(link, idd_conv,i)   
    add_nc(idd_conv,i)

C:\Users\harsh\AppData\Local\Temp/ipykernel_9612/3282793742.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_inp['text'][i] = f.read()


In [8]:
new_inp["cleaned_text"] = ""


In [9]:
#function does preprocessing task 
def clean_it(idd,i):
    #tokenization
    text = idd
    text= regexp_tokenize(text, "[\w']+")
    #normalization
    text = [word for word in text if word.isalpha()]
    #stemming and lemmitizing together
    lemmatizer = WordNetLemmatizer()
    ps = PorterStemmer()
    text = [lemmatizer.lemmatize(word) if word.endswith('e') else ps.stem(word) for word in text]
    #removing stop words
    cleaned_words = [word for word in text if word.lower() not in stop_words]
    #list to string
    cleaned_txt = " ".join(cleaned_words)
    new_inp["cleaned_text"][i] = cleaned_txt
#run through each file  
for i in range(len(new_inp["text"])): 
    idd = new_inp["text"][i]
    clean_it(idd,i)
    #cleaned  for one file have to run for loop tto clean all files and saving file remaninng    

C:\Users\harsh\AppData\Local\Temp/ipykernel_9612/3084925876.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_inp["cleaned_text"][i] = cleaned_txt


In [10]:
# #cleaning
# def clean_text(each_file):
#     cleaned = [word for word in each_file.split(" ") if word not in stoplist]
#     #list_to_text
#     cleaned_file= ' '.join(word for word in cleaned)
#     return cleaned_file
    
# #funnction edits the original file with clean text
# def create_cleaned(write_file,open_file):
#     #edit the original file
#     f = open(open_file,'w', encoding="utf-8")
#     f.write(write_file)
#     f.close()
# #run through each file    
# for i in range(len(inp["URL_ID"][:1])):
#     idd = inp["URL_ID"][i]
#     #variable added to string
#     idd_conv = '%s.txt' % idd
#     org_file = open(idd_conv, encoding="utf-8").read()
#     clean_text(org_file)
#     create_cleaned(cleaned_file, org_file)
#     #cleaned  for one file have to run for loop tto clean all files and saving file remaninng

In [11]:
new_inp["tokens_text"] = ""

In [12]:
def create_tokens(k):
    prp_text = new_inp['cleaned_text'][k]
    tokens = regexp_tokenize(prp_text, "[\w']+")
    new_inp["tokens_text"][k] = tokens
for k in range(len(new_inp["cleaned_text"])):
    create_tokens(k)

C:\Users\harsh\AppData\Local\Temp/ipykernel_9612/2041606934.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_inp["tokens_text"][k] = tokens


In [13]:
pos_org = pd.read_csv('./PN/Pos.txt', names=['pos'])
# pos_org_cleaned = [word for word in pos_org if word.lower() in stop_words]
pos_words= [] 
# pos_org

In [14]:
for element in pos_org['pos']:
    pos_words.append(element.strip())
# pos_words    

In [15]:
neg_org = pd.read_csv('./PN/Neg.txt', names=['neg'])
neg_words= [] 
# neg_org

In [16]:
for element in neg_org['neg']:
    neg_words.append(element.strip())
# neg_words    

In [17]:
new_inp

,URL_ID,URL,text,cleaned_text,tokens_text
0,1,https://insights.blackcoffer.com/how-is-login-...,How is Login Logout Time Tracking for Employee...,login logout Time track employe Office ai blac...,"[login, logout, Time, track, employe, Office, ..."
1,2,https://insights.blackcoffer.com/how-does-ai-h...,How does AI help to monitor Retail Shelf watch...,doe ai monitor retail shelf watch blackcoff in...,"[doe, ai, monitor, retail, shelf, watch, black..."
2,3,https://insights.blackcoffer.com/ai-and-its-im...,AI and its impact on the Fashion Industry - Bl...,ai impact fashion industri blackcoff insight f...,"[ai, impact, fashion, industri, blackcoff, ins..."
3,4,https://insights.blackcoffer.com/how-do-deep-l...,How do deep learning models predict old and ne...,deep learn model predict drug success treat he...,"[deep, learn, model, predict, drug, success, t..."
4,5,https://insights.blackcoffer.com/how-artificia...,How artificial intelligence can boost your pro...,artifici intelligence boost product level blac...,"[artifici, intelligence, boost, product, level..."
...,...,...,...,...,...
164,166,https://insights.blackcoffer.com/data-driven-d...,"Data Driven Dashboards, Foundation and Applica...",data driven dashboard foundat applic industri ...,"[data, driven, dashboard, foundat, applic, ind..."
165,167,https://insights.blackcoffer.com/role-big-data...,Role of Big Data in Academia - Blackcoffer Ins...,Role big data academia blackcoff insight acade...,"[Role, big, data, academia, blackcoff, insight..."
166,168,https://insights.blackcoffer.com/sales-forecas...,Methods for Sales Forecasting in Retail - Blac...,method sale forecast retail blackcoff insight ...,"[method, sale, forecast, retail, blackcoff, in..."
167,169,https://insights.blackcoffer.com/detect-data-e...,Detect Data ExFiltration over the Network - Bl...,detect data exfiltr network blackcoff insight ...,"[detect, data, exfiltr, network, blackcoff, in..."


In [18]:
new_inp['POSITIVE_SCORE'] = ""
new_inp['NEGATIVE_SCORE'] = ""
new_inp['POLARITY SCORE'] = ""
new_inp['SUBJECTIVITY SCORE'] = ""
new_inp['AVG SENTANCE LENGTH'] = ""
new_inp['PERCENTAGE OF COMPLEX WORDS'] = ""
new_inp['FOG INDEX'] = ""
new_inp['AVG NUMBER OF WORDS PER SENTENCE'] = ""
new_inp['COMPLEX WORD COUNT'] = ""
new_inp['WORD COUNT'] = ""
new_inp['SYLLABLE PER WORD'] = ""
new_inp['PERSONAL PRONOUNS'] = ""
new_inp['AVG WORD LENGTH'] = ""

In [19]:
def syllable_count(word):
    word = word.lower()
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
        
    return(count)
pos_score = 0
neg_score = 0
point= 0
for word in new_inp['tokens_text']:
        #count syllable of each word
        syl_count = [syllable_count(word[j])for j in range(len(word))]
        new_inp['SYLLABLE PER WORD'][point] = sum(syl_count)/len(word)
        #greater than 2 syl count
        two_syl_count = sum(each_syl > 2 for each_syl in syl_count)
        new_inp['COMPLEX WORD COUNT'][point] = int(two_syl_count)
        for i in range(len(word)):
            if word[i] in pos_words:
                pos_score +=1
            elif word[i] in neg_words:
                neg_score -=1
        neg_score = neg_score*-1    
        new_inp["POSITIVE_SCORE"][point] = int(pos_score)
        if int(neg_score)<0:
            new_inp["NEGATIVE_SCORE"][point]=-1*int(neg_score)
        else:
            new_inp["NEGATIVE_SCORE"][point]=int(neg_score)
        
        new_inp['POLARITY SCORE'][point] = (pos_score-neg_score)/ ((pos_score + neg_score) + 0.000001)     
        new_inp['SUBJECTIVITY SCORE'][point] = (pos_score + neg_score)/ (len(new_inp["tokens_text"][0]) + 0.000001)
        #######pronouns
        pers_word= 0
        for text in new_inp['tokens_text']:    
            for w in text:
                if w in ["i", "we", "my", "our", "us", "I", "WE", "MY", "OUR", "US"]:
                    pers_word+=1    
            new_inp['PERSONAL PRONOUNS']  = pers_word
        point+=1
        
        
        
        
        

C:\Users\harsh\AppData\Local\Temp/ipykernel_9612/1750871912.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_inp['SYLLABLE PER WORD'][point] = sum(syl_count)/len(word)
C:\Users\harsh\AppData\Local\Temp/ipykernel_9612/1750871912.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_inp['COMPLEX WORD COUNT'][point] = int(two_syl_count)
C:\Users\harsh\AppData\Local\Temp/ipykernel_9612/1750871912.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [20]:
new_inp

,URL_ID,URL,text,cleaned_text,tokens_text,POSITIVE_SCORE,NEGATIVE_SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTANCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,1,https://insights.blackcoffer.com/how-is-login-...,How is Login Logout Time Tracking for Employee...,login logout Time track employe Office ai blac...,"[login, logout, Time, track, employe, Office, ...",17,5,0.545455,0.05641,,,,,55,,1.794872,0,
1,2,https://insights.blackcoffer.com/how-does-ai-h...,How does AI help to monitor Retail Shelf watch...,doe ai monitor retail shelf watch blackcoff in...,"[doe, ai, monitor, retail, shelf, watch, black...",34,2,0.888889,0.092308,,,,,60,,1.879552,0,
2,3,https://insights.blackcoffer.com/ai-and-its-im...,AI and its impact on the Fashion Industry - Bl...,ai impact fashion industri blackcoff insight f...,"[ai, impact, fashion, industri, blackcoff, ins...",86,17,0.669903,0.264103,,,,,175,,1.905284,0,
3,4,https://insights.blackcoffer.com/how-do-deep-l...,How do deep learning models predict old and ne...,deep learn model predict drug success treat he...,"[deep, learn, model, predict, drug, success, t...",94,17,1.441558,0.197436,,,,,45,,1.900415,0,
4,5,https://insights.blackcoffer.com/how-artificia...,How artificial intelligence can boost your pro...,artifici intelligence boost product level blac...,"[artifici, intelligence, boost, product, level...",137,30,0.640719,0.428205,,,,,102,,1.84345,0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,166,https://insights.blackcoffer.com/data-driven-d...,"Data Driven Dashboards, Foundation and Applica...",data driven dashboard foundat applic industri ...,"[data, driven, dashboard, foundat, applic, ind...",4119,59,1.029064,10.410256,,,,,53,,1.866944,0,
165,167,https://insights.blackcoffer.com/role-big-data...,Role of Big Data in Academia - Blackcoffer Ins...,Role big data academia blackcoff insight acade...,"[Role, big, data, academia, blackcoff, insight...",4143,75,0.964438,10.815385,,,,,130,,1.958005,0,
166,168,https://insights.blackcoffer.com/sales-forecas...,Methods for Sales Forecasting in Retail - Blac...,method sale forecast retail blackcoff insight ...,"[method, sale, forecast, retail, blackcoff, in...",4168,63,1.030694,10.525641,,,,,122,,2.0,0,
167,169,https://insights.blackcoffer.com/detect-data-e...,Detect Data ExFiltration over the Network - Bl...,detect data exfiltr network blackcoff insight ...,"[detect, data, exfiltr, network, blackcoff, in...",4182,102,0.952381,10.984615,,,,,93,,1.972318,0,


In [21]:
 new_inp['total_sent'] = ""

In [22]:
turk = 0
for text in new_inp['text']:
    new_inp['total_sent'][turk] = len(sent_tokenize(text))
#     print(number_of_sentences)
    turk+=1

C:\Users\harsh\AppData\Local\Temp/ipykernel_9612/2843828069.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_inp['total_sent'][turk] = len(sent_tokenize(text))


In [23]:
bug = 0
for text in new_inp['cleaned_text']:
    #original_text
    #     print(text)
    text= regexp_tokenize(text, "[\w'\w.]+")        
    #total char
    L = 0
    L =[len(w) for w in text]
#     print(L)
    total_char = sum(L)
#     print(text)
    new_inp['WORD COUNT'][bug] = int(len(text))
#     print(total_words_para)
    text = " ".join(text)
# avg word length
    new_inp['AVG WORD LENGTH'][bug] = round(total_char/new_inp['WORD COUNT'][bug], 2)
    new_inp['AVG NUMBER OF WORDS PER SENTENCE'][bug] = int(new_inp['WORD COUNT'][bug])/int(new_inp['total_sent'][bug])
#     print(avg_word_in_sent)
    ####percent of complex
    new_inp['PERCENTAGE OF COMPLEX WORDS'][bug] = new_inp['COMPLEX WORD COUNT'][bug]/new_inp['total_sent'][bug]
#     print(percent_complex_words)
    new_inp['FOG INDEX'][bug] = 0.4 * (new_inp['AVG NUMBER OF WORDS PER SENTENCE'][bug]+ new_inp['PERCENTAGE OF COMPLEX WORDS'][bug])
#     print(fog_index)
    new_inp['AVG SENTANCE LENGTH'][bug] = new_inp['AVG NUMBER OF WORDS PER SENTENCE'][bug]
    bug+=1

C:\Users\harsh\AppData\Local\Temp/ipykernel_9612/3659731271.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_inp['WORD COUNT'][bug] = int(len(text))
C:\Users\harsh\AppData\Local\Temp/ipykernel_9612/3659731271.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_inp['AVG WORD LENGTH'][bug] = round(total_char/new_inp['WORD COUNT'][bug], 2)
C:\Users\harsh\AppData\Local\Temp/ipykernel_9612/3659731271.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [24]:
new_inp

,URL_ID,URL,text,cleaned_text,tokens_text,POSITIVE_SCORE,NEGATIVE_SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTANCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH,total_sent
0,1,https://insights.blackcoffer.com/how-is-login-...,How is Login Logout Time Tracking for Employee...,login logout Time track employe Office ai blac...,"[login, logout, Time, track, employe, Office, ...",17,5,0.545455,0.05641,15.6,2.2,7.12,15.6,55,390,1.794872,0,5.71,25
1,2,https://insights.blackcoffer.com/how-does-ai-h...,How does AI help to monitor Retail Shelf watch...,doe ai monitor retail shelf watch blackcoff in...,"[doe, ai, monitor, retail, shelf, watch, black...",34,2,0.888889,0.092308,12.310345,2.068966,5.751724,12.310345,60,357,1.879552,0,5.88,29
2,3,https://insights.blackcoffer.com/ai-and-its-im...,AI and its impact on the Fashion Industry - Bl...,ai impact fashion industri blackcoff insight f...,"[ai, impact, fashion, industri, blackcoff, ins...",86,17,0.669903,0.264103,13.025974,2.272727,6.119481,13.025974,175,1003,1.905284,0,5.91,77
3,4,https://insights.blackcoffer.com/how-do-deep-l...,How do deep learning models predict old and ne...,deep learn model predict drug success treat he...,"[deep, learn, model, predict, drug, success, t...",94,17,1.441558,0.197436,15.0625,2.8125,7.15,15.0625,45,241,1.900415,0,6.04,16
4,5,https://insights.blackcoffer.com/how-artificia...,How artificial intelligence can boost your pro...,artifici intelligence boost product level blac...,"[artifici, intelligence, boost, product, level...",137,30,0.640719,0.428205,10.793103,1.758621,5.02069,10.793103,102,626,1.84345,0,5.68,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,166,https://insights.blackcoffer.com/data-driven-d...,"Data Driven Dashboards, Foundation and Applica...",data driven dashboard foundat applic industri ...,"[data, driven, dashboard, foundat, applic, ind...",4119,59,1.029064,10.410256,10.931818,1.204545,4.854545,10.931818,53,481,1.866944,0,5.89,44
165,167,https://insights.blackcoffer.com/role-big-data...,Role of Big Data in Academia - Blackcoffer Ins...,Role big data academia blackcoff insight acade...,"[Role, big, data, academia, blackcoff, insight...",4143,75,0.964438,10.815385,10.732394,1.830986,5.025352,10.732394,130,762,1.958005,0,5.98,71
166,168,https://insights.blackcoffer.com/sales-forecas...,Methods for Sales Forecasting in Retail - Blac...,method sale forecast retail blackcoff insight ...,"[method, sale, forecast, retail, blackcoff, in...",4168,63,1.030694,10.525641,15.315789,3.210526,7.410526,15.315789,122,582,2.0,0,6.0,38
167,169,https://insights.blackcoffer.com/detect-data-e...,Detect Data ExFiltration over the Network - Bl...,detect data exfiltr network blackcoff insight ...,"[detect, data, exfiltr, network, blackcoff, in...",4182,102,0.952381,10.984615,10.140351,1.631579,4.708772,10.140351,93,578,1.972318,0,5.81,57


In [25]:
final_out = new_inp

In [26]:
final_out.drop(columns=['text', 'cleaned_text', 'tokens_text', 'total_sent'])

,URL_ID,URL,POSITIVE_SCORE,NEGATIVE_SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTANCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,1,https://insights.blackcoffer.com/how-is-login-...,17,5,0.545455,0.05641,15.6,2.2,7.12,15.6,55,390,1.794872,0,5.71
1,2,https://insights.blackcoffer.com/how-does-ai-h...,34,2,0.888889,0.092308,12.310345,2.068966,5.751724,12.310345,60,357,1.879552,0,5.88
2,3,https://insights.blackcoffer.com/ai-and-its-im...,86,17,0.669903,0.264103,13.025974,2.272727,6.119481,13.025974,175,1003,1.905284,0,5.91
3,4,https://insights.blackcoffer.com/how-do-deep-l...,94,17,1.441558,0.197436,15.0625,2.8125,7.15,15.0625,45,241,1.900415,0,6.04
4,5,https://insights.blackcoffer.com/how-artificia...,137,30,0.640719,0.428205,10.793103,1.758621,5.02069,10.793103,102,626,1.84345,0,5.68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164,166,https://insights.blackcoffer.com/data-driven-d...,4119,59,1.029064,10.410256,10.931818,1.204545,4.854545,10.931818,53,481,1.866944,0,5.89
165,167,https://insights.blackcoffer.com/role-big-data...,4143,75,0.964438,10.815385,10.732394,1.830986,5.025352,10.732394,130,762,1.958005,0,5.98
166,168,https://insights.blackcoffer.com/sales-forecas...,4168,63,1.030694,10.525641,15.315789,3.210526,7.410526,15.315789,122,582,2.0,0,6.0
167,169,https://insights.blackcoffer.com/detect-data-e...,4182,102,0.952381,10.984615,10.140351,1.631579,4.708772,10.140351,93,578,1.972318,0,5.81


In [27]:
final_out.to_csv('Output.csv', index=False)